In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
import os
import shutil

C:\Users\Leapfrog\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:

# Define paths
original_dataset_dir = r"dataset\original_data"
base_dir = r"dataset"

# Create directories for training and validation splits
train_dir = os.path.join(base_dir, 'train')
os.makedirs(train_dir, exist_ok=True)
validation_dir = os.path.join(base_dir, 'validation')
os.makedirs(validation_dir, exist_ok=True)


# Iterate through each category (class) in the raw-img directory
for category in os.listdir(original_dataset_dir):
    category_dir = os.path.join(original_dataset_dir, category)
    
    # Ensure category_dir is a directory
    if os.path.isdir(category_dir):
        images = [f for f in os.listdir(category_dir) if os.path.isfile(os.path.join(category_dir, f))]
        
        # Debugging: Print out the number of images found
        print(f"Processing category: {category} - {len(images)} images found")

        # Skip this category if there are no images
        if len(images) == 0:
            print(f"No images found in {category}, skipping this category.")
            continue
        
        # Split the images into training and validation sets
        train_images, validation_images = train_test_split(images, test_size=0.2, random_state=42)
        
        
        # Create category directories in train and validation folders
        train_category_dir = os.path.join(train_dir, category)
        os.makedirs(train_category_dir, exist_ok=True)
        validation_category_dir = os.path.join(validation_dir, category)
        os.makedirs(validation_category_dir, exist_ok=True)
        
        # Copy the images
        for image in train_images:
            src = os.path.join(category_dir, image)
            dst = os.path.join(train_category_dir, image)
            shutil.copy2(src, dst)
        
        for image in validation_images:
            src = os.path.join(category_dir, image)
            dst = os.path.join(validation_category_dir, image)
            shutil.copy2(src, dst)

print("Data successfully split into training and validation sets!")

Processing category: cat - 210 images found
Processing category: dog - 210 images found
Processing category: elephant - 210 images found
Processing category: tiger - 210 images found
Data successfully split into training and validation sets!


In [3]:
# Define parameters
image_size = (224, 224)
batch_size = 16

# Image data generator for loading images
datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)

# Load dataset from directory
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='validation'
)


Found 539 images belonging to 4 classes.
Found 32 images belonging to 4 classes.


In [4]:
# Load the VGG16 model without the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of VGG16
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of VGG16
x = base_model.output
x = Flatten(name='feature_extract')(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()



Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ feature_extract (Flatten)            │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │      25,691,136 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 40,409,924 (154.15 MB)

 Trainable params: 25,695,236 (98.02 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

Epoch 1/10


C:\Users\Leapfrog\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


34/34 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step - accuracy: 0.4696 - loss: 7.9992 - val_accuracy: 0.7188 - val_loss: 1.2253
Epoch 2/10
12/34 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.7957 - loss: 0.9567

In [ ]:
import matplotlib.pyplot as plt


# Plotting accuracy
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

# Plotting loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='upper right')

plt.show()


In [ ]:
# Extract features from the training data using the trained model
feature_extractor = Model(inputs=model.input, outputs=model.layers[-3].output)  # Output from the dense layer before softmax

train_features = feature_extractor.predict(train_generator)
train_labels = train_generator.classes

# Flatten the features for SVM
train_features = train_features.reshape(train_features.shape[0], -1)

# Extract features from the validation data
validation_features = feature_extractor.predict(validation_generator)
validation_labels = validation_generator.classes

# Flatten the features for SVM
validation_features = validation_features.reshape(validation_features.shape[0], -1)


In [ ]:
kernel_list = ['linear','rbf','poly','sigmoid']

for kernel in kernel_list:
    # Initialize the SVM classifier
    svm = SVC(kernel=kernel,degree=1,C=0.7)

    # Train the SVM model
    svm.fit(train_features, train_labels)
    # Predict using the SVM classifier
    y_pred = svm.predict(validation_features)

    # Evaluate the performance
    accuracy = accuracy_score(validation_labels, y_pred)
    print(f'Accuracy using {kernel} kernel: {accuracy * 100:.2f}%')

    # Print a classification report
    print(classification_report(validation_labels, y_pred, target_names=validation_generator.class_indices.keys()))